# FPCUP Demo
Author: Olivier Burggraaff, Universiteit Leiden, 2024

This notebook demonstrates some of the functionality of the Python package developed within the FPCUP project.

## Installation and setup
This section will briefly explain how to use Jupyter notebooks and how to install the FPCUP package.
Experienced users can skip ahead.

### Jupyter notebooks
Each block of code or text in this notebook is contained within a "cell".
To run a code cell, click on the cell to select it, then press Shift + Enter on your keyboard, or click the "Run" button in the toolbar at the top.
The code will execute, and the the results (text, data tables, visualisations) will be displayed immediately below the cell.
You can edit and re-run cells as needed to see updated results. To add a new cell, click the "+" button in the toolbar.
You can choose between a code cell or a Markdown cell for text, explanations, or equations.
Save your progress regularly by clicking the disk icon in the toolbar or by pressing Ctrl + S.
This ensures that all your work, including code, outputs, and notes, is preserved.

In [ ]:
print("Hello world")

One of the best features of Jupyter notebooks is the ability to create and plot figures within one interface, allowing for easy testing and customisation without having to fiddle with file formats and the like.
The following sections will contain some examples of figures that can be made with the FPCUP package.
For now, let's test that everything has been installed properly with a simple plot.
Run the following cell; if the output is a plot showing a parabola, you can continue.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
x = np.linspace(-10, 10, 1000)  # 1000 points between -10 and 10, inclusive
y = x**2
plt.plot(x, y)

### Setting up FPCUP
The easiest way to download the FPCUP package is to clone it from GitHub: https://github.com/burggraaff/fpcup.
If you are running this notebook locally, then you have probably already cloned it.

Once downloaded, you can install fpcup using the `pip` programme.
You can run the following command in your terminal or command prompt:
```
pip install .
```
This will install the package and all of its requirements.

Try running the following cell to see if the installation worked.
Please note that this may take up to a minute; if you see `[*]` next to the cell, that means it is still running.

If you get an output like    
```Succesfully imported fpcup version x.y.z from my/file/location/fpcup/__init__.py.```    
then the installation was successful!

In [ ]:
import fpcup
fpcup.test()

## Data

### BRP
Data from the 2019–2024 BRPs (Basisregistratie Percelen) can be imported as follows:

In [ ]:
# Setup
year = 2020  # Try changing this to a different year
brp = fpcup.io.load_brp(year)

# Example output
brp

The BRP contains geographical data, which we can use to draw figures.
For example, the following cell will generate a figure showing the locations of all plots where barley, maize, sorghum, soy, or wheat was grown in the selected year.
Please note that this may take up to a minute; if you see `[*]` next to the cell, that means it is still running.

In [ ]:
# Plot
fpcup.plotting.brp_map_crop(brp)

Using the `province` keyword argument, we can also select only one area, as shown below.
Try changing `"Zuid-Holland"` to different province names.

In [ ]:
# Plot
province = fpcup.geo.process_input_province("Zuid-Holland") 
fpcup.plotting.brp_map_crop(brp, province=province)

#### Interactive plots with Folium
In addition to the static plots shown above, we can also create interactive plots overlaid on OpenStreetMap using the Folium package.

In [ ]:
# Interactive plot
province = fpcup.geo.process_input_province("Zuid-Holland") 
fpcup.plotting.brp_map_crop_interactive(brp, province=province)

## Running WOFOST simulations
WOFOST (WOrld FOod STudies) is a crop model developed at Wageningen University & Research.
The FPCUP package provides an easy-to-use interface for running simulations in WOFOST (using its PCSE implementation).

In this section, we will look at how to set up a single WOFOST run with pre-determined parameters; how to run an ensemble of WOFOST models to determine uncertainties; and how to run WOFOST for multiple sites.

### Single WOFOST run
First, we will set up a single WOFOST run using one set of parameters.
We begin by defining the site location, soil type, management calendar (sowing date, harvest type).

In [ ]:
crop = fpcup.crop.crops["barley"]
soildata = fpcup.soil.soil_types["ec1"]

year = 2022
agromanagement = crop.agromanagement_first_sowingdate(year)  # First sowing date of the year

latitude = 52.1650
longitude = 4.4642

We can use the NASA Power database to retrieve weather data.
The data are downloaded from the internet and stored on your computer, meaning it may take a while the first time per site but will be much faster afterwards.

In [ ]:
weatherdata = fpcup.weather.load_weather_data_NASAPower((latitude, longitude))

Finally, we combine the run data and run the model:

In [ ]:
rundata = fpcup.model.RunData(soildata=soildata, weatherdata=weatherdata, agromanagement=agromanagement, latitude=latitude, longitude=longitude)
output = fpcup.model.run_pcse_single(rundata)

We can examine the model output.
First, we can tabulate the time series, i.e. the values of different outputs over time.
Note that some values may be missing (NaN), for example for dates prior to the sowing date.

In [ ]:
output.timeseries

We can also look at a summary describing the crop state at the end of the run.
This table will only have one row because we only did one run; this type of output will become more informative later, when we do larger ensemble runs.

In [ ]:
output.summary

#### Using the BRP
We can also use the BRP to retrieve coordinates and crop data automatically for one plot.

In [ ]:
# Setup BRP
year = 2020
brp = fpcup.io.load_brp(year)
index = 1  # Change this to any index in the BRP table. Try some values and see what happens!

# Select plot
brp_plot = brp.iloc[index]
brp_plot

In [ ]:
# Setup WOFOST
crop_name = brp_plot["crop"]
crop = fpcup.crop.select_crop(crop_name)
agromanagement = crop.agromanagement_first_sowingdate(year)  # First sowing date of the year

soildata = fpcup.soil.soil_types["ec1"]  # In the future, it will be possible to retrieve the soil type specific to this plot

coordinates = brp_plot["latitude"], brp_plot["longitude"]  # Center of the plot
weatherdata = fpcup.weather.load_weather_data_NASAPower(coordinates)

We again combine the run data, now using the special BRP class, and run the model.

In [ ]:
rundata = fpcup.model.RunDataBRP(brpdata=brp_plot, brpyear=year, soildata=soildata, weatherdata=weatherdata, agromanagement=agromanagement)
output = fpcup.model.run_pcse_single(rundata)

In [ ]:
output.summary

### Running a WOFOST ensemble across multiple locations

More interesting than a single WOFOST run is the ability to do multiple runs at the same time.
This can be used to simulate multiple locations at the same time, such as different plots in the BRP or random sites within one province.
In this example, we will run WOFOST for all barley-growing plots in the province of Zuid-Holland.

In [ ]:
# Setup BRP
year = 2020
brp = fpcup.io.load_brp(year)

# Select plots
crop_species = "barley"
province = fpcup.geo.process_input_province("Zuid-Holland")
brp_plots = fpcup.io.query_brp(brp, province=province, crop_species=crop_species)
brp_plots

In [ ]:
# Setup WOFOST
crop = fpcup.crop.select_crop(crop_species)
agromanagement = crop.agromanagement_first_sowingdate(year)  # First sowing date of the year

soildata = fpcup.soil.soil_types["ec1"]  # In the future, it will be possible to retrieve the soil type specific to each plot

# Combine constants, variables
constants = {"agromanagement": agromanagement, "soildata": soildata}

Rather than keeping all of the results in working memory, they are saved to files which can be accessed later.
For this, we must also set up a directory.

In [ ]:
# Setup output folder
output_dir = fpcup.io.Path("outputs/demo/")

Now we can run WOFOST models for all sites.
Please note that this can take a very long time depending on the number of sites; a progress bar will indicate progress as the models run.

In [ ]:
# Run WOFOST
model_statuses = fpcup.model.run_pcse_brp_ensemble(brp_plots, year=year, run_data_constants=constants, output_dir=output_dir)

Having saved the outputs to file, we can now load them again for inspection and analysis.
Note that this will also load the outputs from any previous runs that were saved to the same directory.
If there are a lot of files, this may take a while, as shown by the progress bar(s).

In [ ]:
summary = fpcup.io.load_combined_ensemble_summary_geo(output_dir)

Since we have multiple outputs, the tabulated summary becomes more interesting:

In [ ]:
summary

Additionally, we can plot the results in several ways.
First, a general summary showing the distribution of some key outputs across the different runs, as histograms and geospatially.
By default, the geospatial plots are aggregated to a hexagonal grid using Uber's H3 system.

In [ ]:
fpcup.plotting.plot_wofost_summary(summary)

This plot can also be made for a single province:

In [ ]:
province = fpcup.geo.process_input_province("Zuid-Holland") 
fpcup.plotting.plot_wofost_summary(summary, province=province)

Alternatively, we can aggregate the results by province.
Of course, if the previous cells were only run for a single province, then the aggregate statistics are not very interesting.
Try running through this section several times, for different provinces, and see how the statistics fill up.

In [ ]:
summary_byprovince = fpcup.aggregate.aggregate_province(summary)
fpcup.plotting.plot_wofost_summary_byprovince(summary_byprovince)

### Testing model sensitivity using WOFOST ensembles
Ensembles can also be used to estimate the uncertainty in outputs by using different combinations of input parameters.
Here, we will look at an example using one site and varying one parameter.
First, we select a site from the BRP as before:

In [ ]:
# Setup BRP
year = 2020
brp = fpcup.io.load_brp(year)
index = 1  # Change this to any index in the BRP table. Try some values and see what happens!

# Select plot
brp_plot = brp.iloc[index]
brp_plot

Next, we set the parameters that we want to remain fixed:

In [ ]:
# Setup WOFOST
crop_name = brp_plot["crop"]
crop = fpcup.crop.select_crop(crop_name)
agromanagement = crop.agromanagement_first_sowingdate(year)  # First sowing date of the year

soildata = fpcup.soil.soil_types["ec1"]  # In the future, it will be possible to retrieve the soil type specific to this plot

coordinates = brp_plot["latitude"], brp_plot["longitude"]  # Center of the plot
weatherdata = fpcup.weather.load_weather_data_NASAPower(coordinates)

# Combine constants
constants = {"agromanagement": agromanagement, "soildata": soildata, "weatherdata": weatherdata, "latitude": coordinates[0], "longitude": coordinates[1]}

Now, we set the variable parameters.
In this case, we will look at the effects of varying `WAV`, that is the initial amount of water in the rootable zone in excess of wilting point.
WOFOST allows for `WAV` values between 0 and 50 cm, with a default of 10 cm (which was implicitly used in the previous runs).
You can use the variable `n` to determine how many values of `WAV` will be used.

In [ ]:
n = 101  # Number of values to test
variable_names = ["WAV"]
combined_parameters = fpcup.parameters.generate_ensemble_space(*variable_names, n=n)
variables = {"override": combined_parameters}
variables = fpcup.tools.dict_product(variables)  # Pack the variables into one iterable of dictionaries

# Setup output folder
output_dir = fpcup.io.Path("outputs/demo/WAV/")

In [ ]:
model_statuses = fpcup.model.run_pcse_ensemble(variables, output_dir, run_data_constants=constants)

The outputs can be plotted as follows:

In [ ]:
summary = fpcup.io.load_combined_ensemble_summary_geo(output_dir)
fpcup.plotting.sensitivity_one_crop(summary, crop_name, variable_names[0])

#### Ensembles testing multiple parameters
It is also possible to run an ensemble for multiple variables at the same time, although this exponentially increases the run time.
Here, we will look at an example where both `WAV` (as before) and `RDMSOL` (the maximum rooting depth allowed by the soil) are varied at the same time.
All other parameters are re-used from the previous cells.

In [ ]:
n = 51  # Number of values to test
variable_names = ["WAV", "RDMSOL"]
combined_parameters = fpcup.parameters.generate_ensemble_space(*variable_names, n=n)
variables = {"override": combined_parameters}
variables = fpcup.tools.dict_product(variables)  # Pack the variables into one iterable of dictionaries

# Setup output folder
output_dir = fpcup.io.Path("outputs/demo/RDMSOL-WAV/")

In [ ]:
model_statuses = fpcup.model.run_pcse_ensemble(variables, output_dir, run_data_constants=constants)

#### Multiple sowing dates
As an additional example, we can run WOFOST using fixed or variable parameters for multiple sowing dates, to see the effects this has on crop growth.